In [1]:
import pandas as pd
import numpy as np
from pandas import datetime
import warnings 
warnings.
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import adfuller
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
%matplotlib inline

Using TensorFlow backend.
C:\Users\AnuragDubey\Anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\AnuragDubey\Anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\AnuragDubey\Anaconda3\envs\tfdeeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:

In [2]:
def dateparser(X):
    return datetime.strptime("190"+X,"%Y-%m")

#convert time series into supervised learning problem
def series_to_supervised(data,n_In,n_Out, dropnan= True):
    n_vars =1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(),list()
    # input sequence (t-n, ... t-1)
    for i in range(n_In,0,-1):
        cols.append(df.shift(i))
        names+=["var%d(t-%d)"% (j+1,i) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0,n_Out):
        cols.append(df.shift(-i))
        names+=["var%d(t+%d)"%(j+1,i) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols,axis=1)
    agg.columns= names
    if dropnan:
        agg.dropna(inplace= True)
    return agg
#preparing train test data
def prepareData(series,n_test,n_lag,n_seq):
    rawData = series.values
    rawData =rawData.reshape(len(rawData),1)
    supervised = series_to_supervised(rawData,n_lag,n_seq)
    supervisedVal = supervised.values
    train,test = supervisedVal[0:-n_test],supervisedVal[-n_test:]
    return train,test

In [3]:
#Load data
data= pd.read_csv("C:\\TensorFlow\\LSTM\\shamData.csv",index_col=0, parse_dates=[0],date_parser= dateparser)
train,test = prepareData(data,10,1,3)
print("Train %s, test %s"% (str(train.shape),str(test.shape)))

Train (23, 4), test (10, 4)


In [4]:
test

array([[342.3, 339.7, 440.4, 315.9],
       [339.7, 440.4, 315.9, 439.3],
       [440.4, 315.9, 439.3, 401.3],
       [315.9, 439.3, 401.3, 437.4],
       [439.3, 401.3, 437.4, 575.5],
       [401.3, 437.4, 575.5, 407.6],
       [437.4, 575.5, 407.6, 682. ],
       [575.5, 407.6, 682. , 475.3],
       [407.6, 682. , 475.3, 581.3],
       [682. , 475.3, 581.3, 646.9]])

In [5]:
def persistence(last_ob, n_seq):
    return [last_ob for i in range(n_seq)]
def make_forecasts(train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
    # make forecast
        forecast = persistence(X[-1], n_seq)
        # store the forecast
        forecasts.append(forecast)
    return forecasts

forecasts = make_forecasts(train, test, 1, 3)

In [6]:
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = test[:,(n_lag+i)]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))
evaluate_forecasts(test,forecasts,1,3)

t+1 RMSE: 144.535304
t+2 RMSE: 86.479905
t+3 RMSE: 121.149168


# Multi-Step LSTM Network

Stationary. The data shows an increasing trend that must be removed by differencing.
Scale. The scale of the data must be reduced to values between -1 and 1, the activation function of the LSTM units.

In [18]:
def series_to_supervised(data,n_In,n_Out, dropnan= True):
    n_vars =1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(),list()
    # input sequence (t-n, ... t-1)
    for i in range(n_In,0,-1):
        cols.append(df.shift(i))
        names+=["var%d(t-%d)"% (j+1,i) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0,n_Out):
        cols.append(df.shift(-i))
        names+=["var%d(t+%d)"%(j+1,i) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols,axis=1)
    agg.columns= names
    if dropnan:
        agg.dropna(inplace= True)
    return agg

def difference(data,interval=1):
    diff= list()
    for i in range(interval,len(data)):
        val = data[i]-data[i-interval]
        diff.append(val)
    return pd.Series(diff)

def prepareData(data,n_test,n_leg,n_seq):
    rawVal = data.values
    diffSeries = difference(rawVal,1)
    diffVals = diffSeries.values
    diffVals =diffVals.reshape(len(diffVals),1)
    scaler= MinMaxScaler()
    scaler = scaler.fit(diffVals)
    scaledVal = scaler.fit_transform(diffVals)
    scaledVal = scaledVal.reshape(len(scaledVal),1)
    supervised = series_to_supervised(scaledVal, n_leg, n_seq)
    supervised_values = supervised.values
    # split into train and test sets
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    return scaler, train, test

In [82]:
def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # reshape training into [samples, timesteps, features]
    X,y = train[:,0:n_lag],train[:,n_lag:]
    X= X.reshape(X.shape[0],1,X.shape[1])
    #design network
    model= Sequential()
    model.add(LSTM(n_neurons,batch_input_shape=(n_batch,X.shape[1],X.shape[2]),stateful=True,return_sequences=True))
    model.add(LSTM(2,stateful=True))
    model.add(Dense(y.shape[1]))
    model.compile(loss='mean_squared_error',optimizer='adam')
    # fit the network
    for i in range(nb_epoch):
        model.fit(X,y,epochs=1,batch_size=n_batch,verbose=0,shuffle= False)
        model.reset_states()
    return model

A single forecast can be made with the fit LSTM network by calling model.predict(). Again, the data must be formatted into a 3D array with the format [samples, timesteps, features].

In [76]:
def forcast_lstm(model, X, n_batch):
    X= X.reshape(1,1,len(X))
    #make forecast
    forecast = model.predict(X,batch_size=n_batch)
    #convert to array
    return([x for x in forecast[0,:]])
# evaluate the persistence model
def make_forecast(model,n_batch,train,test,n_lag,n_seq):
    forecasts = list()
    for i in range(len(test)):
        X,y = test[i,0:n_lag],test[i,n_lag:]
        forecast = forcast_lstm(model,X,n_batch)
        forecasts.append(forecast)
    return forecasts
#make forecast
forecasts = make_forecast(model,1,train,test,1,3)

Invert Transforms


After the forecasts have been made, we need to invert the transforms to return the values back into the original scale.
This is needed so that we can calculate error scores and plots that are comparable with other models, like the persistence forecast above.
We can invert the scale of the forecasts directly using the MinMaxScaler object that offers an inverse_transform() function.

We can invert the differencing by adding the value of the last observation (prior months’ shampoo sales) to the first forecasted value, then propagating the value down the forecast.
This is a little fiddly; we can wrap up the behavior in a function name inverse_difference() that takes the last observed value prior to the forecast and the forecast as arguments and returns the inverted forecast.


In [77]:
def inverse_difference(last_obser,forecast):
    #invert first forecast
    inverted= list()
    inverted.append(forecast[0]+last_obser)
    #propogate difference forecast using inverted value
    for i in range(1,len(forecast)):
        inverted.append(forecast[i]+inverted[i-1])
    return inverted
#Putting this together, we can create an inverse_transform() function that works through each forecast,
#first inverting the scale and then inverting the differences, returning forecasts to their original scale.
def inverse_transform(series,forecasts,scaler,n_test):
    inverted = list()
    for i in range(len(forecasts)):
        #create an array for forecast
        forecast = np.array(forecasts[i])
        forecast= forecast.reshape(1,len(forecast))
        #invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0,:]
        #invert differencing
        index = len(series)-n_test+i-1
        last_obser = series.values[index]
        inv_diff= inverse_difference(last_obser,inv_scale)
        inverted.append(inv_diff)
    return inverted

# evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))

In [78]:
series = pd.read_csv("C:\\TensorFlow\\LSTM\\shamData.csv",index_col=0, parse_dates=[0],date_parser= dateparser)
# configure
n_lag=1
n_seq =3
n_test=10
n_epochs=600
n_batch=1
n_neurons=2

# prepare data
scaler, train, test = prepareData(series, n_test, n_lag, n_seq)

C:\Users\AnuragDubey\Anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [83]:
# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)

In [84]:
# make forecasts
forecasts = make_forecast(model, n_batch, train, test, n_lag, n_seq)
# inverse transform forecasts and test
forecasts = inverse_transform(series, forecasts, scaler, n_test+2)

In [85]:
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test+2)
# evaluate forecasts
evaluate_forecasts(actual, forecasts, n_lag, n_seq)

t+1 RMSE: 144.064122
t+2 RMSE: 84.701543
t+3 RMSE: 109.072262
